# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet

#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
+ Formulas for the Bernoulli distribution: https://en.wikipedia.org/wiki/Bernoulli_distribution

In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
import scipy.stats as stats
import pylab

data = pd.io.stata.read_stata('us_job_market_discrimination.dta')

/Users/springboard/miniconda3/envs/tutorial/lib/python2.7/site-packages/matplotlib/__init__.py:872: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


In [2]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4870 entries, 0 to 4869
Data columns (total 65 columns):
id                    4870 non-null object
ad                    4870 non-null object
education             4870 non-null int8
ofjobs                4870 non-null int8
yearsexp              4870 non-null int8
honors                4870 non-null int8
volunteer             4870 non-null int8
military              4870 non-null int8
empholes              4870 non-null int8
occupspecific         4870 non-null int16
occupbroad            4870 non-null int8
workinschool          4870 non-null int8
email                 4870 non-null int8
computerskills        4870 non-null int8
specialskills         4870 non-null int8
firstname             4870 non-null object
sex                   4870 non-null object
race                  4870 non-null object
h                     4870 non-null float32
l                     4870 non-null float32
call                  4870 non-null float32
city        

In [3]:
data.head()

,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,...,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
0,b,1,4,2,6,0,0,0,1,17,...,1,0,1,0,0,0,0,0,0,
1,b,1,3,3,6,0,1,1,0,316,...,1,0,1,0,0,0,0,0,0,
2,b,1,4,1,6,0,0,0,0,19,...,1,0,1,0,0,0,0,0,0,
3,b,1,3,4,6,0,1,0,1,313,...,1,0,1,0,0,0,0,0,0,
4,b,1,3,3,22,0,0,0,0,313,...,1,1,0,0,0,0,0,1,0,Nonprofit


<div class="span5 alert alert-success">
<p>...</p>
</div>

# 1. What test is appropriate for this problem? Does CLT apply?

The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not. As such, the data follows a binomial distribution.

A two-sample t-test is appropriate for this problem. The Central Limit Theorem applies because the sample is independent, random, and n > 30. 

# 2. What are the null and alternate hypotheses?

$H_{o}$ : There is no difference in the rate of callbacks for black-sounding and white-sounding resumes ($P_{1}$ - $P_{2}$ = 0)  

$H_{a}$ : There is a difference in the rate of callbacks for black-sounding and white-sounding resumes ($P_{1}$ - $P_{2}$ != 0)  

α = 0.05

# 3. Compute margin of error, confidence interval, and p-value. 
# Try using both the bootstrapping and the frequentist statistical approaches.

<div class="span5 alert alert-success">
<p>Frequentist Approach</p>
</div>

In [4]:
b = data[data.race=='b']
w = data[data.race=='w']

In [5]:
# Number of black-sounding names
total_b_names = sum(data.race=='b')
print(total_b_names)

# Number of callbacks for black-sounding names
total_b_callbacks = sum(b.call)
print(total_b_callbacks)


# Rate of callbacks for black-sounding names
rate_b = total_b_callbacks/total_b_names
print(rate_b)

2435
157.0
0.064476386037


In [6]:
# Number of white-sounding names
total_w_names = sum(data.race=='w')
print(total_w_names)

# Number of callbacks for white-sounding names
total_w_callbacks = sum(w.call)
print(total_w_callbacks)

# Rate of callbacks for white-sounding names
rate_w = total_w_callbacks/total_w_names
print(rate_w)

2435
235.0
0.0965092402464


In [7]:
# P1 - P2

abs_diff = abs(rate_b - rate_w)
abs_diff

0.032032854209445585

There is a 95% chance that the true population proportion is within d of 0.032. 

__Margin of Error = z * Standard Error__

For a significance level of 5%, the cumulative area under the curve is 97.5% and the corresponding z value is 1.96.

Standard error is calculated by taking the standard deviation of the sampling distribution and dividing it by the square root of n.

In [8]:
# Variance: black-sounding names

# Variance = P1 * (1 - P1) / n

P1 = rate_b
n1 = total_b_names

var_black = P1 * (1 - P1) / n1
print(var_black)

# Variance: white-sounding names

P2 = rate_w
n2 = total_w_names

var_white = P2 * (1 - P2) / n2
print(var_white)

2.47717378565e-05
3.5809119833e-05


In [9]:
# Standard Deviation = sqrt(var_black + var_white)

std_dev = np.sqrt(var_black + var_white)
std_dev

0.0077833705866767544

In [10]:
# Margin of Error = z * Standard Deviation

z = 1.96
margin_error = z * std_dev

print('Margin of Error' + '= ' + str(margin_error))

Margin of Error= 0.0152554063499


In [11]:
# Confidence Interval

lower_range = abs_diff - margin_error
upper_range = abs_diff + margin_error

print('Confidence Interval' + ': ' + str(lower_range) + ' - ' + str(upper_range))

Confidence Interval: 0.0167774478596 - 0.0472882605593


In [12]:
# P-Value: Two-Sample T-Test

b_array = data[data.race=='b'].call
w_array = data[data.race=='w'].call

stats.ttest_ind(b_array, w_array, equal_var=True)

Ttest_indResult(statistic=-4.1147052908617514, pvalue=3.9408021031288859e-05)

<div class="span5 alert alert-success">
<p>Bootstrap Approach</p>
</div>

In [13]:
# Use concatenate function to group desired data together

grouped_b_w = np.concatenate((b_array, w_array))

In [14]:
# Bootstrap replicate function

def bootstrap_replicate(data, func):
    bs_sample = np.random.choice(grouped_b_w, size=len(grouped_b_w))
    return func(bs_sample)

In [15]:
# Generating many bootstrap replicates

def draw_bs_reps(data, func, size=1):
    bs_replicates = np.empty(1000)
    for z in range(1000):
        bs_replicates[z] = bootstrap_replicate(data, func)
        return bs_replicates

In [16]:
# Generating many bootstrap replicates

def draw_bs_reps(data, func, size=1):
    bs_replicates = np.empty(1000)
    for z in range(1000):
        bs_replicates[z] = bootstrap_replicate(data, func)
        return bs_replicates

In [17]:
# Generating bootstrap replicates

def draw_bs_reps(data, func, size=1):
    bs_replicates = np.empty(10000)
    
    for z in range(10000):
        b_replicates = np.sum(np.random.choice(grouped_b_w, size=2435)) 
        w_replicates = np.sum(np.random.choice(grouped_b_w, size=2435))
        bs_replicates[z] = (b_replicates - w_replicates)/2435
    return bs_replicates

In [18]:
bs_replicates = draw_bs_reps(grouped_b_w, np.mean, 1000)

In [19]:
p = np.sum(bs_replicates >= abs_diff)/10000

print('P Value' + ': ' + str(p))

P Value: 0


In [20]:
conf_int = np.percentile(bs_replicates, [2.5, 97.5])
conf_int

array([-0.01519507,  0.01519507])

# 4. Write a story describing the statistical significance in the context or the original problem.

As the p-value = 3.9408021031288859e-05 is lower than the 0.05 significance level, the null hypothesis is rejected for the alternate which states that there is a difference in the rate of callbacks for black-sounding and white-sounding resumes.

# 5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

Race and name can definitely affect one's rate of callbacks, but I do not think this analysis allows for the conclusion that it is the most important factor in callback success. Employers can hold a variety of other biases - education and experience level, for example. 

There are many different ways in which this analysis could be amended. Callback rates for variables such as years of experience, and especially gender, should be investigated in relation to race, though I believe a very interesting analysis could include the variables 'occupspecific' and 'occupbroad' with race as the target variable to determine if there is a significant difference in the rate of callback success for names in niche markets. 